In [1]:
from core.rabagliati_13_analysis import *
import pandas as pd

<Figure size 432x288 with 0 Axes>

In [2]:
df = pd.read_csv('../data/rabagliati_2013_stimuli.csv', sep = '\t')
df = df[~df['target'].isin(['moose', 'mousse'])] #BERT tokenization issues


Formatting the dataframe so we can run BERT through it

In [3]:
df['pos'] = ['n'] * len(df.index)
df['word'] = df['target']
word_pos_pairs = df[['word', 'pos']].drop_duplicates()
word_pos_pairs.to_csv('../data/r_13_for_semcor.csv')

Generating masked language modeling predictions. We replace the target token with a [MASK] and see what BERT predicts when the identity of the word is NOT given.

In [64]:
mask_results = masking_embeds_preds(df)

Inspecting which words have types of both senses that were used in the stimuli vs. only one sense

In [5]:
both_sense_types, one_sense_type = check_semcor_data(word_pos_pairs, df)

In [65]:
len(both_sense_types), both_sense_types

(15,
 ['chicken',
  'fish',
  'herbs',
  'letter',
  'button',
  'bow',
  'line',
  'bat',
  'nail',
  'pitcher',
  'band',
  'sun',
  'son',
  'knight',
  'night'])

In [66]:
len(one_sense_types), one_sense_types

(8, ['lamb', 'turkey', 'duck', 'glasses', 'mouse', 'roll', 'cards', 'bark'])

In [41]:
all_data = generate_sense_predictions(df, mask_results)
sum(all_data['centroid_preds'] == all_data['wn_sense']) / len(all_data.index)

0.6190476190476191

In [47]:
sum(all_data['nn_preds'] == all_data['wn_sense']) / len(all_data.index)

0.6071428571428571

In [53]:
accuracy_table(all_data, 'rel_type')

,rel_type,centroid_acc,nn_acc
0,reg_pol,0.458333,0.500000
1,irreg_pol,0.812500,0.656250
2,hom,0.535714,0.642857


In [61]:
accuracy_table(all_data, "sentence_ctx")

,sentence_ctx,centroid_acc,nn_acc
0,curr,0.595238,0.595238
1,prev,0.642857,0.619048
